In [1]:
import itertools
import numpy as np
import pandas as pd
import os
import sklearn.decomposition
import sys

CURRENT_DIR = os.getcwd()
PARENT_DIR = os.path.dirname(CURRENT_DIR)
sys.path.append(PARENT_DIR)

from FDApy.univariate_functional import UnivariateFunctionalData
from FDApy.irregular_functional import IrregularFunctionalData
from FDApy.multivariate_functional import MultivariateFunctionalData
from FDApy.fpca import UFPCA, MFPCA
from FDApy.local_polynomial import LocalPolynomial

In [2]:
# Load the data into a Pandas Dataframes
precipitation = pd.read_csv(''.join([CURRENT_DIR, '/canadian_precipitation_monthly.csv']), index_col=0)
temperature = pd.read_csv(''.join([CURRENT_DIR, '/canadian_temperature_daily.csv']), index_col=0)

In [3]:
# Create univariate functional data for the precipitation data
argvals = tuple(pd.factorize(precipitation.columns)[0])
values = np.array(precipitation)
monthlyPrec = UnivariateFunctionalData(argvals, values)

argvals is converted into one dimensional list!


In [4]:
# Create univariate functional data for the daily temperature data.
argvals = tuple(pd.factorize(temperature.columns)[0])
values = np.array(temperature)
dailyTemp = UnivariateFunctionalData(argvals, values)

argvals is converted into one dimensional list!


In [5]:
# Create multivariate functional data for the Canadian weather data.
canadWeather = MultivariateFunctionalData([dailyTemp, monthlyPrec])

In [6]:
# Print example of a UnivariateFunctionalData
dailyTemp

Univariate Functional data objects with 35 observations of 1-dimensional support
argvals:
	0, 1, ... , 364	(365 sampling points)
values:
	array of size (35, 365)

In [7]:
# Print example of a MultivariateFunctionalData
canadWeather

Multivariate Functional data objects with 2 funtions:
- Univariate Functional data objects with 35 observations of 1-dimensional support
argvals:
	0, 1, ... , 364	(365 sampling points)
values:
	array of size (35, 365)
- Univariate Functional data objects with 35 observations of 1-dimensional support
argvals:
	0, 1, ... , 11	(12 sampling points)
values:
	array of size (35, 12)

In [8]:
# Accessing the attribut of the classes
monthlyPrec.argvals

[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)]

In [9]:
# Number of observations for an object
monthlyPrec.nObs()

35

In [10]:
# Number of sampling points for an object
canadWeather.nObsPoint()

[[365], [12]]

In [11]:
# Dimension of the domain of observations
dailyTemp.dimension()

1

In [12]:
# Extract observations from UnivariateFunctionalData
dailyTemp[3:6]

Univariate Functional data objects with 3 observations of 1-dimensional support
argvals:
	0, 1, ... , 364	(365 sampling points)
values:
	array of size (3, 365)

In [13]:
# Extract functions from MultivariateFunctionalData
canadWeather[0]

Convert data into one dimensional list.


Multivariate Functional data objects with 1 funtions:
- Univariate Functional data objects with 35 observations of 1-dimensional support
argvals:
	0, 1, ... , 364	(365 sampling points)
values:
	array of size (35, 365)

In [14]:
# Convert UnivariateFunctionalData into IrregularFunctionalData.
dailyTemp.asIrregularFunctionalData()

Irregular Functional data objects with 35 observations of 1-dimensional support
argvals:
	Values in 0 ... 364.
values:
	Values in -34.8 ... 22.8.
There are 365 - 365 sampling points per observation.

In [15]:
# Convert UnivariateFunctionalData into MultivariateFunctionalData.
monthlyPrec.asMultivariateFunctionalData()

Multivariate Functional data objects with 1 funtions:
- Univariate Functional data objects with 35 observations of 1-dimensional support
argvals:
	0, 1, ... , 11	(12 sampling points)
values:
	array of size (35, 12)

In [16]:
# Add another function to a MultivariateFunctionalData
#canadWeather.add(dailyTemp)

In [17]:
# Perform a FPCA.
fpca = UFPCA(n_components=0.99)
fpca.fit(dailyTemp)

In [18]:
# Compute the scores of the dailyTemp data into the eigenfunctions basis.
dailyTemp_proj = fpca.transform(dailyTemp)

In [19]:
# Perform multivariate FPCA
mfpca = MFPCA(n_components=0.99)
mfpca.fit(canadWeather)

In [22]:
# Compute the scores of the canadWeather data into the multivariate eigenfunctions basis.
mfpca.transform(canadWeather)

array([[-132.48456476,  -77.77545745],
       [-168.70048174, -104.18429566],
       [-158.04106556,  -91.94306115],
       [-138.11521537, -119.10523175],
       [-178.85501317,  -85.81799073],
       [-197.52881828,  -86.30724967],
       [-230.15348486,  113.73939051],
       [-226.77058301,  -43.89060779],
       [-227.52720778,  -27.62067118],
       [-215.58484702,  -62.89517604],
       [-203.31663119,  -64.97725181],
       [-214.72641541, -102.09562926],
       [-217.7462841 ,  -95.86563278],
       [-189.01091483, -124.69534457],
       [-188.94381897, -125.75862082],
       [-214.16869348,  -31.74833332],
       [-254.03313189,  -29.40850019],
       [-256.06586462,   21.37826315],
       [-248.52060364,  155.588718  ],
       [-237.33150184,  -35.42407872],
       [-248.78093538,    4.42121652],
       [-279.99680732,   85.20772067],
       [-204.59795704,  -28.19707921],
       [-173.13073977,  -62.46331304],
       [-171.68687645, -153.96001441],
       [-104.03209185, -1

In [20]:
# Smooth the data
#points = tuple(i / 10 for i in range(min(dailyTemp.argvals[0]), 10*(max(dailyTemp.argvals[0]) + 1), 1))
#dailyTempSmooth = dailyTemp.smooth(points, bandwith=5)